In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.optimizers.schedules import PiecewiseConstantDecay
import os
import pandas as pd

In [ ]:
import yaml
import logging
from datetime import datetime

# YAML config
try:
    with open(r".\config.yaml", "r") as f:
        config = yaml.safe_load(f)
except Exception as e:
    raise

# Logger
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(name)s - %(funcName)s - %(message)s",
    filename=config["log_dir"] +
    f"{datetime.now().strftime("%Y-%m-%d_%H-%M-%S")}.log",
    filemode="w"
)
logger = logging.getLogger(__name__)

logger.info("Config file and logger setup completed.")

In [ ]:
config["image_size"] = tuple(config["image_size"])

In [ ]:
# Helper Functions
def load_data(data_dir, subset):
    """
    Generates training and validation data using ImageDataGenerator.

    Args:
        data_dir (str): Path to the directory containing the insect images.
        subset (str): "training" or "validation".

    Returns:
        tf.data.Dataset: A TensorFlow dataset for the specified subset.
    """
    if subset == "training":
        logger.info("Generating training data...")
        dataset = tf.keras.preprocessing.image.ImageDataGenerator(
            rescale=1 / 255,
            rotation_range=20,
            width_shift_range=0.1,
            height_shift_range=0.1,
            shear_range=0.1,
            zoom_range=0.2,
            horizontal_flip=True,
            preprocessing_function=preprocess_input
        ).flow_from_directory(directory=data_dir, target_size=(200, 200), subset="training")
    elif subset == "validation":
        logger.info("Generating validation data...")
        dataset = tf.keras.preprocessing.image.ImageDataGenerator(
            rescale=1 / 255,
            validation_split=0.2,
            preprocessing_function=preprocess_input
        ).flow_from_directory(directory=data_dir, target_size=(200, 200), subset="validation")
    else:
        raise ValueError("Invalid subset.  Must be 'training' or 'validation'")
    logger.info("Data generation complete.")
    return dataset

In [ ]:
def build_model():
    """Builds the CNN model using VGG16 as a base."""
    logger.info("Building the model...")
    base_model = tf.keras.applications.VGG16(
        input_shape=config["image_size"], weights="imagenet", include_top=False)
    base_model.trainable = False
    logger.info("Model built successfully.")
    x = tf.keras.layers.Flatten()(base_model.output)
    x = tf.keras.layers.Dense(config["class_count"], activation="softmax")(x)
    model = Model(base_model.input, x)
    logger.info("Model built successfully.")
    return model

In [ ]:
def train_model(model, train_dataset, validation_dataset, batch_size, epochs):
    """Trains the model."""
    logger.info("Starting model training...")
    optimizer = Adam(learning_rate=config["learning_rate"])
    model.compile(optimizer=optimizer,
                  loss="categorical_crossentropy", metrics=["accuracy"])
    # Use callbacks for early stopping.
    callback = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=3)
    r = model.fit(train_dataset, epochs=epochs, validation_data=validation_dataset, steps_per_epoch=train_dataset.n // batch_size,
                  validation_steps=validation_dataset.n // batch_size, callbacks=callback)
    logger.info("Model training complete.")
    return r

In [ ]:
def save_model(model, save_path):
    """Saves the trained model."""
    logger.info(f"Saving model to {save_path}...")
    model.save(save_path)
    logger.info("Model saved successfully.")

In [ ]:
def load_model(save_path):
    """Loads the trained model."""
    logger.info(f"Loading model from {save_path}...")
    loaded_model = tf.keras.models.load_model(save_path)
    logger.info("Model loaded successfully.")
    return loaded_model

In [ ]:
def predict_image(model, image_path):
    """
    Predicts the insect class for a given image.
    """
    logger.info(f"Predicting image: {image_path}")
    img = image.load_img(image_path, target_size=config["image_size"])
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array.astype("float32") / 255.0  # Normalize
    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction)
    labels = {0: "Butterfly", 1: "Dragonfly",
              2: "Grasshopper", 3: "Ladybird", 4: "Mosquito"}
    logger.info(
        f"Predicted class: {predicted_class}, Label: {labels[predicted_class]}")
    return predicted_class, labels[predicted_class]

In [ ]:
# Build the model
logger.info("Building the model...")
model = build_model()

# Load and generate training/validation data
logger.info("Loading data...")
train_generator = load_data(config["data_path"], "training")
valid_generator = load_data(config["data_path"], "validation")
logger.info("Data loaded.")

# Train the model
logger.info("Starting model training...")
r = train_model(model, train_generator, valid_generator,
                config["batch_size"], config["epochs"])

In [ ]:
# Save the model
logger.info("Saving the model...")
save_model(model, config["save_path"])

# Load the model
logger.info("Loading the model...")
loaded_model = load_model(config["save_path"])

# Example Prediction
logger.info("Running example prediction...")
image_path = "path"
predicted_class, predicted_label = predict_image(loaded_model, image_path)
print(f"Predicted Class: {predicted_class}, Label: {predicted_label}")